# Test the Stream Index

## Check if index is up!

In [1]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "loGu88Z",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "u6p1OlcKS8K0ulvvGBdVKw",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['localhost'], http_auth=('elastic', 'changeme'), port=9200)

## Index operations

#### Delete index, delete tweets by id and query

In [7]:
es.indices.delete(index='stream', ignore=[400, 404])

{u'acknowledged': True}

In [ ]:
# es.delete_by_query(index='stream', doc_type='tweet', body={
#     'query': {
#         "match_all" : {}
#     }
# })

#### Count elements in index

In [11]:
es.count(index='stream', doc_type='tweet')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 4}

#### GET all docs from index

In [12]:
res = es.search(index="stream", doc_type='tweet', size=100, body={"query": {"match_all": {}}})
# [x['_source'] for x in res['hits']['hits']]
for r in res['hits']['hits']:
    print r
    print 

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'880200119806824448', u'_source': {u'username': u'Ktanasushiwok', u'lang': u'es', u'text_categories': [u' sushi'], u'city': u'Candelaria', u'text': u'Ya estamos listos para entregarte como siempre, lo mejor del #Sushi y el #Wok en #CaliCo\u2026 https://t.co/hODbwmPyhF', u'img_categories': None, u'hashtags': [u'#sushi', u'#wok', u'#calico\u2026'], u'img_flag': True, u'timestamp_ms': u'1498691036546', u'bounding_box': {u'type': u'Polygon', u'coordinates': [[[-76.584528, 3.266369], [-76.584528, 3.461995], [-76.407632, 3.461995], [-76.407632, 3.266369]]]}, u'coords': [-76.53565396, 3.40648448], u'id_str': u'880200119806824448', u'country': u'Colombia', u'id': 880200119806824448, u'created_at_day': 20170628, u'created_at_datetime': u'2017-06-28T23:03:56', u'media_url': None}, u'_index': u'stream'}

{u'_score': 1.0, u'_type': u'tweet', u'_id': u'880199694093348864', u'_source': {u'username': u'Gabyoc31', u'lang': u'fr', u'text_categories': [u' po

#### GET specific tweets

In [ ]:
es.get(index='stream', doc_type='tweet_snippet', id=877436391956860928)

In [ ]:
import json
from index.preprocess_tweet import get_image_category
results = es.search(index='stream', doc_type='tweet_snippet', size=100, body={
  'query': {
    'match': {
      'img_category': 'Sushi',
#       'day': 20170618,
#         'sort': [
#         {'committed_date': {'order': 'desc'}}
#       ],
     }
  }
})

# print results
# print len(results["hits"]["hits"])

#### TEST image classifier

In [ ]:
# test some of the urls from the indexed tweets
# by reclassifying it with the IMG classifier hosted on our machine
request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
for tweet in results["hits"]["hits"]:
    img_url = tweet["_source"]["media_url"]
    res = requests.get(request_string + img_url)

    print tweet["_source"]["text"]
    print img_url
    print res.content
    print json.loads(res.text)["guessed"]
    print get_image_category(img_url)
    print 
    print

In [ ]:
# import requests
# import json

# response = requests.get('http://test.tripbuilder.isti.cnr.it:5000/sync')
# json_data = json.loads(response.text)
# json_data